In [1]:
import sys


sys.path = [p for p in sys.path if 'controller' not in p]

# Insere o controller certo do Webots no início
sys.path.insert(0, '/Applications/Webots.app/Contents/lib/controller/python') #Mila

import os

import math
import random as pyrandom
import numpy as np
from controller import Supervisor, Lidar

from controller import Robot


In [2]:
try:
    from controllers.utils import cmd_vel
except ImportError:
    print("Warning: controllers.utils.cmd_vel not found. Using dummy function.")
    def cmd_vel(supervisor, lv, av):
        # Dummy implementation for testing without the utility file
        left_motor = supervisor.getDevice("left wheel motor")
        right_motor = supervisor.getDevice("right wheel motor")
        wheel_radius = 0.02 # Assuming a standard wheel radius
        axle_track = 0.0565 # Assuming a standard axle track
        left_velocity = (lv - av * axle_track / 2.0) / wheel_radius
        right_velocity = (lv + av * axle_track / 2.0) / wheel_radius
        left_motor.setVelocity(left_velocity)
        right_motor.setVelocity(right_velocity)

In [3]:

# --- Configuration ---
ROBOT_NAME = "e-puck"
ROBOT_RADIUS = 0.035  # meters

# Tunnel clearance relative to robot diameter (2 * ROBOT_RADIUS)
MIN_CLEARANCE_FACTOR = 2.5
MAX_CLEARANCE_FACTOR = 4.0

# Minimum required clearance for the robot to pass an obstacle
MIN_ROBOT_CLEARANCE_FACTOR = 2.1
MIN_ROBOT_CLEARANCE = ROBOT_RADIUS * MIN_ROBOT_CLEARANCE_FACTOR

# Minimum distance between obstacles
MIN_OBSTACLE_DISTANCE_FACTOR = 4.0
MIN_OBSTACLE_DISTANCE = ROBOT_RADIUS * MIN_OBSTACLE_DISTANCE_FACTOR

# Number of obstacles to place in the tunnel
NUM_OBSTACLES = 2


BASE_SPEED = 0.1
BASE_WALL_LENGTH = 1.0
WALL_THICKNESS = 0.01
WALL_HEIGHT = 0.07
MIN_CURVE_ANGLE = math.radians(10)
MAX_CURVE_ANGLE = math.radians(90)
CURVE_SUBDIVISIONS = 30 # Increased subdivisions for smoother curves
TIMEOUT_DURATION = 45.0
NUM_CURVES = 3

# Map boundaries (example values, adjust to your world limits)
MAP_X_MIN, MAP_X_MAX = -2.0, 2.0
MAP_Y_MIN, MAP_Y_MAX = -2.0, 2.0

# Small overlap factor to ensure no gaps in curves
OVERLAP_FACTOR = 1.01 # Adjusted overlap factor based on user feedback


In [4]:
class TunnelBuilder:
    def __init__(self, supervisor):
        self.supervisor = supervisor
        self.root_children = supervisor.getRoot().getField("children")
        self.base_wall_distance = 0
        self.walls = []
        self.segments = []
        self.right_walls = []
        self.left_walls = []
        self.obstacles = []



    def create_wall(self, pos, rot, size, wall_type=None):
        wall = f"""Solid {{
            translation {pos[0]} {pos[1]} {pos[2]}
            rotation {rot[0]} {rot[1]} {rot[2]} {rot[3]}
            children [
                Shape {{
                    appearance Appearance {{
                        material Material {{
                            diffuseColor 1 0 0
                        }}
                    }}
                    geometry Box {{
                        size {size[0]} {size[1]} {size[2]}
                    }}
                }}
            ]
        }}"""
        self.root_children.importMFNodeFromString(-1, wall)
        if wall_type == 'left':
            self.left_walls.append((pos, rot, size))
        elif wall_type == 'right':
            self.right_walls.append((pos, rot, size))
        elif wall_type == 'obstacle':
            self.obstacles.append((pos, rot, size))


    def build_tunnel(self, num_curves, angle_range=None, clearance=None):
        if clearance is not None:
            tunnel_clearance_factor = clearance
        else:
            tunnel_clearance_factor = pyrandom.uniform(MIN_CLEARANCE_FACTOR, MAX_CLEARANCE_FACTOR)
    
        self.base_wall_distance = ROBOT_RADIUS * tunnel_clearance_factor
        print(f"Building tunnel with clearance factor: {tunnel_clearance_factor:.2f}")
    
        angle_min, angle_max = angle_range if angle_range else (MIN_CURVE_ANGLE, MAX_CURVE_ANGLE)

        num_curves = min(num_curves, NUM_CURVES)
        length = BASE_WALL_LENGTH * (1 + pyrandom.uniform(-0.15, 0.15))
        segment_length = length / (num_curves + 1)

        T = np.eye(4)
        start_pos = T[:3, 3].copy()
        walls = []
        segments_data = []

        if not self._within_bounds(T, segment_length):
            return None, None, 0
        segment_start_pos = T[:3, 3].copy()
        self._add_straight(T, segment_length, walls)
        segment_end_pos = T[:3, 3].copy()
        segment_heading = math.atan2(T[1, 0], T[0, 0])
        segments_data.append((segment_start_pos, segment_end_pos, segment_heading, segment_length))

        for _ in range(num_curves):
            angle = pyrandom.uniform(angle_min, angle_max) * pyrandom.choice([1, -1])

            if not self._within_bounds_after_curve(T, angle, segment_length):
                break
            prev_end_pos = T[:3, 3].copy()
            self._add_curve(T, angle, segment_length, walls, segments_data)
            new_start = segments_data[-CURVE_SUBDIVISIONS][0]  # início da curva recém-adicionada
            gap = np.linalg.norm(prev_end_pos - new_start)
            if gap > 1e-3:
                print(f"[WARNING] Discontinuity between curve segments: gap = {gap:.3f}")
            if not self._within_bounds(T, segment_length):
                break
            segment_start_pos = T[:3, 3].copy()
            self._add_straight(T, segment_length, walls)
            segment_end_pos = T[:3, 3].copy()
            segment_heading = math.atan2(T[1, 0], T[0, 0])
            segments_data.append((segment_start_pos, segment_end_pos, segment_heading, segment_length))

        end_pos = T[:3, 3].copy()
        self._add_obstacles(segments_data, walls)
        self.walls = walls
        self.segments = segments_data
        return start_pos, end_pos, len(walls)

    def point_to_segment_distance(self, A, B, P):
        AP = P - A
        AB = B - A
        t = np.clip(np.dot(AP, AB) / np.dot(AB, AB), 0.0, 1.0)
        closest_point = A + t * AB
        return np.linalg.norm(P - closest_point)

    def is_robot_near_centerline(self, robot_position):
        """
        Checks if the robot is close enough to the center line of the tunnel,
        considering all segments (straight + curves).
        """
        robot_xy = np.array(robot_position[:2])
        threshold = self.base_wall_distance + ROBOT_RADIUS
    
        for start, end, _, _ in self.segments:
            dist = self.point_to_segment_distance(start[:2], end[:2], robot_xy)
            if dist <= threshold:
                return True
        return False #vir aqui 


    
        robot_xy = np.array(robot_position[:2])
        right_dir = np.array([math.cos(heading - math.pi/2), math.sin(heading - math.pi/2)])
        left_dir = np.array([math.cos(heading + math.pi/2), math.sin(heading + math.pi/2)])
    
        def check_walls(walls, direction):
            for pos, _, _ in walls:
                wall_xy = np.array(pos[:2])
                to_wall = wall_xy - robot_xy
                dist = np.linalg.norm(to_wall)
                dot = np.dot(to_wall / (dist + 1e-6), direction)
                if dot > 0.7:
                    expected = self.base_wall_distance
                    if abs(dist - expected) < 0.05:
                        return True
            return False
    
        def check_obstacles(obstacles):
            for pos, _, size in obstacles:
                wall_xy = np.array(pos[:2])
                dist = np.linalg.norm(wall_xy - robot_xy)
                # Tolerância baseada na largura do obstáculo
                if dist < size[1] / 2 + ROBOT_RADIUS + 0.02:
                    return True
            return False
    
        near_right = check_walls(self.right_walls, right_dir)
        near_left = check_walls(self.left_walls, left_dir)
        near_obstacle = check_obstacles(self.obstacles)
    
        inside = (near_right and near_left) or near_obstacle
        #print(f"[DEBUG] Inside tunnel: {inside} | Right: {near_right}, Left: {near_left}, Obstacle: {near_obstacle}")
        return inside





    def _add_straight(self, T, length, walls):
        # Calculate the heading of the current segment
        heading = math.atan2(T[1, 0], T[0, 0])
        # Add walls on both sides of the centerline
        for side in [-1, 1]:
            # Calculate the wall's center position:
            # Start at current T's position, move half the length along T's x-axis (forward),
            # then move self.base_wall_distance along T's y-axis (sideways), and half height up.
            pos = T[:3, 3] + T[:3, 0] * (length / 2) + T[:3, 1] * (side * self.base_wall_distance) + np.array([0, 0, WALL_HEIGHT / 2])
            # Rotation is around the z-axis based on the heading
            rot = (0, 0, 1, heading)
            # Size of the wall (length, thickness, height)
            size = (length, WALL_THICKNESS, WALL_HEIGHT)
            wall_type = 'left' if side == -1 else 'right'
            self.create_wall(pos, rot, size, wall_type=wall_type)
            walls.append((pos, rot, size)) # Append wall details to the walls list
        # Update T to the end of the straight segment
        T[:3, 3] += T[:3, 0] * length


    def _add_curve(self, T, angle, segment_length, walls, segments_data):
        step = angle / CURVE_SUBDIVISIONS
        centerline_step_length = segment_length / CURVE_SUBDIVISIONS
        r_centerline = self.base_wall_distance
        r_inner_edge = r_centerline - WALL_THICKNESS / 2.0
        r_outer_edge = r_centerline + WALL_THICKNESS / 2.0

        for _ in range(CURVE_SUBDIVISIONS):
            T_start_step = T.copy()
            segment_start = T[:3, 3].copy()

            T_mid_step = T_start_step @ self._rotation_z(step / 2)
            T_mid_step[:3, 3] += T_mid_step[:3, 0] * (centerline_step_length / 2)
            heading = math.atan2(T_mid_step[1, 0], T_mid_step[0, 0])
            rot = (0, 0, 1, heading)

            for side in [-1, 1]:
                wall_length = r_inner_edge * abs(step) if side == -1 else r_outer_edge * abs(step)
                wall_length += OVERLAP_FACTOR * WALL_THICKNESS
                pos = T_start_step[:3, 3] + T_start_step[:3, 1] * (side * self.base_wall_distance) + np.array([0, 0, WALL_HEIGHT / 2]) + T_start_step[:3, 0] * (wall_length / 2)
                size = (wall_length, WALL_THICKNESS, WALL_HEIGHT)
                wall_type = 'left' if side == -1 else 'right'
                self.create_wall(pos, rot, size, wall_type=wall_type)
                walls.append((pos, rot, size))
                

            T[:] = T_start_step @ self._rotation_z(step)
            T[:3, 3] += T[:3, 0] * centerline_step_length

            segment_end = T[:3, 3].copy()
            segment_heading = math.atan2(T[1, 0], T[0, 0])
            segments_data.append((segment_start, segment_end, segment_heading, centerline_step_length))
            #print(f"[CURVE SEGMENT] start: {segment_start}, end: {segment_end}, heading: {math.degrees(segment_heading):.1f}°")


    def _add_obstacles(self, straight_segments_data, walls):
        """
        Place NUM_OBSTACLES perpendicular walls into the middle straight segments.
        straight_segments_data: list of (start_pos, end_pos, heading, length)
        walls: list to append (pos, rot, size) tuples for cleanup/tracking
        """
        # skip entrance & exit straights
        segments = straight_segments_data[1:-1]
        if not segments:
            print("Not enough segments for obstacles.")
            return

        used = set()
        placed_positions = []

        # half-width of tunnel from centerline
        tunnel_half = ROBOT_RADIUS * MIN_CLEARANCE_FACTOR
        # obstacle span across tunnel, leaving MIN_ROBOT_CLEARANCE on the other side
        obstacle_length = 2 * tunnel_half - MIN_ROBOT_CLEARANCE - WALL_THICKNESS

        for _ in range(NUM_OBSTACLES):
            # pick an unused segment index
            choices = [i for i in range(len(segments)) if i not in used]
            if not choices:
                break
            idx = pyrandom.choice(choices)
            used.add(idx)

            start, end, heading, seg_len = segments[idx]
            # choose a point 20–80% along the straight
            d = pyrandom.uniform(0.2 * seg_len, 0.8 * seg_len)
            dir_vec = np.array([math.cos(heading), math.sin(heading), 0.0])
            pos = np.array(start) + dir_vec * d

            # pick inner/outer side
            side = pyrandom.choice([-1, +1])
            perp = np.array([-dir_vec[1], dir_vec[0], 0.0])
            shift = side * (MIN_ROBOT_CLEARANCE / 2 + obstacle_length / 2)
            pos += perp * shift
            pos[2] = WALL_HEIGHT / 2.0

            # no extra rotation—box X-axis is already perpendicular to the corridor
            rot = (0.0, 0.0, 1.0, heading)
            # size: X = thickness along path, Y = span across path
            size = (WALL_THICKNESS, obstacle_length, WALL_HEIGHT)

            # avoid clustering
            if any(np.linalg.norm(pos[:2] - p) < MIN_OBSTACLE_DISTANCE
                   for p in placed_positions):
                print("Skipping obstacle—too close to another.")
                continue

            # create and record
            wall_type = 'left' if side == -1 else 'right'
            self.create_wall(pos, rot, size, wall_type='obstacle')


            walls.append((pos, rot, size))
            placed_positions.append(pos[:2].copy())

    def _translation(self, x, y, z):
        # Helper function to create a translation matrix
        M = np.eye(4)
        M[:3, 3] = [x, y, z]
        return M

    def _rotation_z(self, angle):
        # Helper function to create a rotation matrix around the z-axis
        c, s = math.cos(angle), math.sin(angle)
        R = np.eye(4)
        R[0, 0], R[0, 1] = c, -s
        R[1, 0], R[1, 1] = s, c
        return R

    def _within_bounds(self, T, length):
        # Check if the end point of a straight segment is within map bounds
        end = T[:3, 3] + T[:3, 0] * length
        return MAP_X_MIN <= end[0] <= MAP_X_MAX and MAP_Y_MIN <= end[1] <= MAP_Y_MAX

    def _within_bounds_after_curve(self, T, angle, seg_len):
        # Check if the end point after a curved segment is within map bounds
        tempT = T.copy()
        step = angle / CURVE_SUBDIVISIONS
        centerline_step_length = seg_len / CURVE_SUBDIVISIONS
        for _ in range(CURVE_SUBDIVISIONS):
            tempT = tempT @ self._rotation_z(step)
            tempT[:3, 3] += tempT[:3, 0] * centerline_step_length
        end = tempT[:3, 3]
        return MAP_X_MIN <= end[0] <= MAP_X_MAX and MAP_Y_MIN <= end[1] <= MAP_Y_MAX

    def is_robot_inside_tunnel(self, robot_position, heading):
        robot_xy = np.array(robot_position[:2])
        right_dir = np.array([math.cos(heading - math.pi/2), math.sin(heading - math.pi/2)])
        left_dir = np.array([math.cos(heading + math.pi/2), math.sin(heading + math.pi/2)])
    
        def check_walls(walls, direction):
            for pos, _, _ in walls:
                wall_xy = np.array(pos[:2])
                to_wall = wall_xy - robot_xy
                dist = np.linalg.norm(to_wall)
                dot = np.dot(to_wall / (dist + 1e-6), direction)
                if dot > 0.7:
                    expected = self.base_wall_distance
                    if abs(dist - expected) < 0.05:
                        return True
            return False
    
        def check_obstacles(obstacles):
            for pos, _, size in obstacles:
                wall_xy = np.array(pos[:2])
                dist = np.linalg.norm(wall_xy - robot_xy)
                # Tolerância baseada na largura do obstáculo
                if dist < size[1] / 2 + ROBOT_RADIUS + 0.02:
                    return True
            return False
    
        near_right = check_walls(self.right_walls, right_dir)
        near_left = check_walls(self.left_walls, left_dir)
        near_obstacle = check_obstacles(self.obstacles)
    
        inside = (near_right and near_left) or near_obstacle
        #print(f"[DEBUG] Inside tunnel: {inside} | Right: {near_right}, Left: {near_left}, Obstacle: {near_obstacle}")
        return inside







In [5]:


class SimulationManager:
    def __init__(self, supervisor):
        self.supervisor = supervisor
        self.timestep = int(self.supervisor.getBasicTimeStep())
        self.robot = self.supervisor.getFromDef(ROBOT_NAME)
        if self.robot is None:
            raise ValueError(f"Robot with DEF name '{ROBOT_NAME}' not found.")
        self.translation = self.robot.getField("translation")
        self.lidar = self.supervisor.getDevice("lidar")
        self.lidar.enable(self.timestep)
        self.stats = {'total_collisions': 0, 'successful_runs': 0, 'failed_runs': 0}
        
    def run_experiment(self, num_runs):
        # Run the experiment for the specified number of runs
        for run in range(num_runs):
            print(f"--- Starting Run {run + 1} ---")
            # Build a new random tunnel
            # Pass the supervisor to the TunnelBuilder constructor
            tunnel_builder = TunnelBuilder(self.supervisor)
            start_pos, end_pos, walls_added = tunnel_builder.build_tunnel(NUM_CURVES)

            # If tunnel building failed (out of bounds), skip this run
            if start_pos is None:
                print("Tunnel out of bounds, skipping run.")
                continue

            # Place the robot at the start of the tunnel and reset physics
            self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
            self.robot.resetPhysics()

            # Get wheel motors and set to infinite position control (velocity mode)
            left = self.supervisor.getDevice("left wheel motor")
            right = self.supervisor.getDevice("right wheel motor")
            left.setPosition(float('inf'))
            right.setPosition(float('inf'))
            left.setVelocity(0) # Start with zero velocity
            right.setVelocity(0)

            collision_count = 0
            # Flag to prevent counting multiple collisions for a single contact event
            flag = False
            start_time = self.supervisor.getTime() # Record start time for timeout

            # Simulation loop
            while self.supervisor.step(self.timestep) != -1:
                # Check for timeout
                if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                    print("Timeout")
                    break

                # Get Lidar data
                data = self.lidar.getRangeImage()
                # Process Lidar data using the new logic
                lv, av = self._process_lidar(data)
                # Apply velocity commands to the robot
                cmd_vel(self.supervisor, lv, av)

                # Get current robot position
                pos = np.array(self.translation.getSFVec3f())

                # Simple collision detection based on distance from centerline
                # This collision detection might need refinement depending on the tunnel path
                # It assumes the robot should stay within BASE_WALL_DISTANCE from the centerline
                # A more robust collision detection would check for actual contact with wall nodes.
                # Need to get the current BASE_WALL_DISTANCE from the tunnel_builder instance
                current_base_wall_distance = tunnel_builder.base_wall_distance
                current_distance_from_center = abs(pos[1]) # Use y-coordinate for distance from centerline
                # The collision detection logic here is still based on the overall tunnel width.
                # Detecting collisions with the new smaller obstacles requires a different approach,
                # ideally using Webots' collision detection events or checking proximity to obstacle nodes.
                # For now, the existing collision check remains, but it won't accurately count collisions with the new obstacles.
                print(f"[DEBUG] Distância do centro: {current_distance_from_center:.3f} | Limite permitido: {current_base_wall_distance - ROBOT_RADIUS:.3f}")

                if current_distance_from_center > current_base_wall_distance - ROBOT_RADIUS:
                    if not flag:
                        print(f"⚡ [ALERTA] Robô saiu do túnel! Posição: {pos[:2]}")
                        flag = True
                elif current_distance_from_center <= current_base_wall_distance - ROBOT_RADIUS:
                    flag = False



                # Check if the robot has reached the end of the tunnel
                if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                    print(f"Reached end in {self.supervisor.getTime()-start_time:.1f}s")
                    break

            # Remove the generated walls after the run
            self._remove_walls(walls_added)

            # Update statistics
            # The collision count here might not be accurate due to the simple collision detection
            self.stats['total_collisions'] += collision_count
            # A run is successful if there were no collisions (based on current detection) and the robot reached the end
            if collision_count == 0 and end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                 self.stats['successful_runs'] += 1
            else:
                 self.stats['failed_runs'] += 1
            print(f"Run {run + 1} finished with {collision_count} collisions.")


        # Print the final summary after all runs
        self._print_summary()

    def _process_lidar_with_params(self, dist_values: [float], distP=10.0, angleP=7.0) -> (float, float):

        """
        Robot control logic based on Lidar data.
        Adapted from IRI - TP2 - Ex 4 by Gonçalo Leão.
        """
        # Assuming direction 1 for wall following (e.g., right wall)
        # You might need to adjust this or add logic to determine the direction
        # based on the tunnel structure or robot's initial position/goal.
        direction: int = 1

        maxSpeed: float = 0.1
        distP: float = 10.0  # Proportional gain for distance error
        angleP: float = 7.0  # Proportional gain for angle error
        wallDist: float = 0.1 # Desired distance to the wall

        # Find the angle of the ray that returned the minimum distance
        size: int = len(dist_values)
        if size == 0:
            # Handle case where lidar data is empty
            return 0.0, 0.0

        min_index: int = 0
        if direction == -1:
            min_index = size - 1
        for i in range(size):
            idx: int = i
            if direction == -1:
                idx = size - 1 - i
            # Ensure distance is valid (greater than 0) before considering it minimum
            if dist_values[idx] < dist_values[min_index] and dist_values[idx] > 0.0:
                min_index = idx
            # If the current min_index has an invalid distance, find the next valid one
            elif dist_values[min_index] <= 0.0 and dist_values[idx] > 0.0:
                 min_index = idx


        angle_increment: float = 2*math.pi / (size - 1) if size > 1 else 0.0
        angleMin: float = (size // 2 - min_index) * angle_increment
        distMin: float = dist_values[min_index]

        # Get distances from specific directions
        distFront: float = dist_values[size // 2] if size > 0 else float('inf')
        distSide: float = dist_values[size // 4] if size > 0 and size // 4 < size else float('inf') if (direction == 1) else dist_values[3*size // 4] if size > 0 and 3*size // 4 < size else float('inf')
        distBack: float = dist_values[0] if size > 0 else float('inf')


        # Prepare message for the robot's motors
        linear_vel: float
        angular_vel: float

        # print("distMin", distMin) # Commented out to reduce console output
        # print("angleMin", angleMin*180/math.pi) # Commented out

        # Decide the robot's behavior
        if math.isfinite(distMin):
            # Check for potential unblocking scenario (stuck)
            if distFront < 1.25*wallDist and (distSide < 1.25*wallDist or distBack < 1.25*wallDist):
                # print("UNBLOCK") # Commented out
                # Turn away from the detected obstacles
                angular_vel = direction * -1 * maxSpeed # Use maxSpeed for turning velocity
                linear_vel = 0 # Stop linear movement while unblocking
            else:
                # print("REGULAR") # Commented out
                # Calculate angular velocity based on distance and angle errors
                # Error 1: Difference between minimum distance and desired wall distance
                # Error 2: Difference between angle of minimum distance and desired angle (pi/2 for side wall)
                angular_vel = direction * distP * (distMin - wallDist) + angleP * (angleMin - direction * math.pi / 2)
                # print("angular_vel", angular_vel, " wall comp = ", direction * distP * (distMin - wallDist), ", angle comp = ", angleP * (angleMin - direction * math.pi / 2)) # Commented out

            # Adjust linear velocity based on front distance
            if distFront < wallDist:
                # If obstacle is very close in front, stop and turn
                # print("TURN") # Commented out
                linear_vel = 0
                # Angular velocity is already calculated above
            elif distFront < 2 * wallDist or distMin < wallDist * 0.75 or distMin > wallDist * 1.25:
                # If obstacle is somewhat close or not at desired wall distance, slow down
                # print("SLOW") # Commented out
                linear_vel = 0.5 * maxSpeed
                # Angular velocity is already calculated above
            else:
                # Otherwise, cruise at max speed
                # print("CRUISE") # Commented out
                linear_vel = maxSpeed
                # Angular velocity is already calculated above
        else:
            # If no finite minimum distance (e.g., open space), wander
            # print("WANDER") # Commented out
            # Use numpy.random.normal for random angular velocity
            angular_vel = np.random.normal(loc=0.0, scale=1.0) * maxSpeed # Scale random value by maxSpeed
            # print("angular_vel", angular_vel) # Commented out
            linear_vel = maxSpeed # Continue moving forward while wandering

        # Clamp angular velocity to a reasonable range if needed (optional but good practice)
        # angular_vel = max(-maxSpeed, min(maxSpeed, angular_vel))


        return linear_vel, angular_vel

    def _remove_walls(self, count):
        # Remove the last 'count' children from the root node
        children = self.supervisor.getRoot().getField("children")
        for _ in range(count):
            children.removeMF(-1)

    def _print_summary(self):
        # Print the final summary of the experiment
        print("\n=== Final Results ===")
        print(f"Successful runs: {self.stats['successful_runs']}")
        print(f"Failed runs: {self.stats['failed_runs']}")
        print(f"Total collisions: {self.stats['total_collisions']}")
        total_runs = self.stats['successful_runs'] + self.stats['failed_runs']
        if total_runs > 0:
             print(f"Success rate: {self.stats['successful_runs'] / total_runs * 100:.1f}%")
        else:
             print("No runs completed.")



#________________ Genetic algorithms______________________

    def _process_lidar(self, dist_values: [float], distP=10.0, angleP=7.0) -> (float, float):

        """
        Robot control logic based on Lidar data, with tunable distP and angleP parameters.
        Adapted for use with Genetic Algorithm optimization.
        """
        direction: int = 1  # Assuming right wall following
    
        maxSpeed: float = 0.1
        wallDist: float = 0.1  # Desired distance to the wall
    
        size: int = len(dist_values)
        if size == 0:
            return 0.0, 0.0  # No data available, stay still
    
        min_index: int = 0
        if direction == -1:
            min_index = size - 1
        for i in range(size):
            idx: int = i
            if direction == -1:
                idx = size - 1 - i
            if dist_values[idx] < dist_values[min_index] and dist_values[idx] > 0.0:
                min_index = idx
            elif dist_values[min_index] <= 0.0 and dist_values[idx] > 0.0:
                min_index = idx
    
        angle_increment: float = 2 * math.pi / (size - 1) if size > 1 else 0.0
        angleMin: float = (size // 2 - min_index) * angle_increment
        distMin: float = dist_values[min_index]
    
        distFront: float = dist_values[size // 2] if size > 0 else float('inf')
        distSide: float = dist_values[size // 4] if (size > 0 and size // 4 < size) else float('inf') if (direction == 1) else dist_values[3 * size // 4] if (size > 0 and 3 * size // 4 < size) else float('inf')
        distBack: float = dist_values[0] if size > 0 else float('inf')
    
        linear_vel: float
        angular_vel: float
    
        if math.isfinite(distMin):
            if distFront < 1.25 * wallDist and (distSide < 1.25 * wallDist or distBack < 1.25 * wallDist):
                angular_vel = direction * -1 * maxSpeed
                linear_vel = 0
            else:
                angular_vel = direction * distP * (distMin - wallDist) + angleP * (angleMin - direction * math.pi / 2)
    
            if distFront < wallDist:
                linear_vel = 0
            elif distFront < 2 * wallDist or distMin < wallDist * 0.75 or distMin > wallDist * 1.25:
                linear_vel = 0.5 * maxSpeed
            else:
                linear_vel = maxSpeed
        else:
            angular_vel = np.random.normal(loc=0.0, scale=1.0) * maxSpeed
            linear_vel = maxSpeed
    
        
        return linear_vel, angular_vel

    #comportamento de um par aleatório, sem ser os níveis fácil, médio, difícil
    def run_experiment_with_params(self, distP: float, angleP: float) -> float:
        total_fitness = 0.0
        difficulties = ['easy', 'medium', 'hard']
    
        for difficulty in difficulties:
            print(f"\n--- Simulação com dificuldade: {difficulty.upper()} ---")
    
            # Obter os parâmetros fixos da dificuldade
            num_curves, angle_range, clearance = get_difficulty_settings(difficulty)
    
            # Criar túnel
            builder = TunnelBuilder(self.supervisor)
            start_pos, end_pos, walls_added = builder.build_tunnel(
                num_curves=num_curves,
                angle_range=angle_range,
                clearance=clearance
            )
    
            if start_pos is None:
                print(f"⚠️ Túnel fora dos limites ({difficulty}). Fitness parcial: -1000.")
                total_fitness += -1000.0
                continue
    
            # Reposicionar robô
            self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
            self.robot.resetPhysics()
    
            # Inicializar motores
            left = self.supervisor.getDevice("left wheel motor")
            right = self.supervisor.getDevice("right wheel motor")
            left.setPosition(float('inf'))
            right.setPosition(float('inf'))
            left.setVelocity(0)
            right.setVelocity(0)
    
            # Inicializar variáveis de simulação
            grace_period = 2.0
            off_tunnel_events = 0
            flag_off_tunnel = False
            start_time = self.supervisor.getTime()
            timeout_occurred = False
            goal_reached = False
            distance_traveled_inside = 0.0
            previous_pos = np.array(self.translation.getSFVec3f())
            distance_traveled = 0.0
            last_pos = previous_pos.copy()
    
            while self.supervisor.step(self.timestep) != -1:
                if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                    timeout_occurred = True
                    print("⏰ TIME OUT")
                    break
    
                data = self.lidar.getRangeImage()
                lv, av = self._process_lidar_with_params(data, distP, angleP)
                cmd_vel(self.supervisor, lv, av)
    
                pos = np.array(self.translation.getSFVec3f())
                distance_traveled += np.linalg.norm(pos[:2] - last_pos[:2])
                last_pos = pos.copy()
                rot = self.robot.getField("rotation").getSFRotation()
                heading = rot[3] if rot[2] >= 0 else -rot[3]
    
                inside_by_geometry = builder.is_robot_near_centerline(pos)
                inside_by_lidar = builder.is_robot_inside_tunnel(pos, heading)
                inside_tunnel = inside_by_geometry or inside_by_lidar
    
                if inside_tunnel:
                    distance_step = np.linalg.norm(pos[:2] - previous_pos[:2])
                    distance_traveled_inside += distance_step
                    flag_off_tunnel = False
                else:
                    if not flag_off_tunnel and self.supervisor.getTime() - start_time > grace_period:
                        off_tunnel_events += 1
                        flag_off_tunnel = True
                        print(f"⚡ [ALERTA] Robô saiu do túnel ({difficulty}): {pos[:2]}")
    
                previous_pos = pos
    
                if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                    goal_reached = True
                    break
    
            # Apagar paredes do túnel antes da próxima dificuldade
            self._remove_walls(walls_added)
    
            # Calcular fitness desta dificuldade
            fitness = 0.0
            total_tunnel_length = sum([seg[3] for seg in builder.segments])
            percent_traveled = distance_traveled / total_tunnel_length if total_tunnel_length > 0 else 0
    
            if percent_traveled >= 0.8:
                fitness += 500
            if goal_reached and not timeout_occurred:
                fitness += 1000
    
            fitness -= 2000 * off_tunnel_events
            fitness += 100 * distance_traveled_inside
    
            print(f"🎯 Fitness para {difficulty}: {fitness:.2f}")
            total_fitness += fitness
    
        # Média das 3 dificuldades
        average_fitness = total_fitness / len(difficulties)
        print(f"\n🔚 Média final do fitness: {average_fitness:.2f}")
        return average_fitness


    def run_on_existing_tunnel(self, distP, angleP, builder, start_pos, end_pos):
        self.translation.setSFVec3f([start_pos[0], start_pos[1], ROBOT_RADIUS])
        self.robot.resetPhysics()
    
        left = self.supervisor.getDevice("left wheel motor")
        right = self.supervisor.getDevice("right wheel motor")
        left.setPosition(float('inf'))
        right.setPosition(float('inf'))
        left.setVelocity(0)
        right.setVelocity(0)
    
        grace_period = 2.0
        off_tunnel_events = 0
        flag_off_tunnel = False
        start_time = self.supervisor.getTime()
        timeout_occurred = False
        goal_reached = False
        distance_traveled_inside = 0.0
        previous_pos = np.array(self.translation.getSFVec3f())
        distance_traveled = 0.0
        last_pos = previous_pos.copy()
    
        while self.supervisor.step(self.timestep) != -1:
            if self.supervisor.getTime() - start_time > TIMEOUT_DURATION:
                timeout_occurred = True
                break
    
            data = self.lidar.getRangeImage()
            lv, av = self._process_lidar_with_params(data, distP, angleP)
            cmd_vel(self.supervisor, lv, av)
    
            pos = np.array(self.translation.getSFVec3f())
            distance_traveled += np.linalg.norm(pos[:2] - last_pos[:2])
            last_pos = pos.copy()
            rot = self.robot.getField("rotation").getSFRotation()
            heading = rot[3] if rot[2] >= 0 else -rot[3]
    
            inside_by_geometry = builder.is_robot_near_centerline(pos)
            inside_by_lidar = builder.is_robot_inside_tunnel(pos, heading)
            inside_tunnel = inside_by_geometry or inside_by_lidar
    
            if inside_tunnel:
                distance_step = np.linalg.norm(pos[:2] - previous_pos[:2])
                distance_traveled_inside += distance_step
                flag_off_tunnel = False
            else:
                if not flag_off_tunnel and self.supervisor.getTime() - start_time > grace_period:
                    off_tunnel_events += 1
                    flag_off_tunnel = True
    
            previous_pos = pos
    
            if end_pos is not None and np.linalg.norm(pos[:2] - end_pos[:2]) < 0.1:
                goal_reached = True
                break
    
        total_tunnel_length = sum([seg[3] for seg in builder.segments])
        percent_traveled = distance_traveled / total_tunnel_length if total_tunnel_length > 0 else 0
    
        fitness = 0.0
        if percent_traveled >= 0.8:
            fitness += 500
        if goal_reached and not timeout_occurred:
            fitness += 1000
        fitness -= 2000 * off_tunnel_events
        fitness += 100 * distance_traveled_inside
    
        return fitness
    


In [6]:
"""
if __name__ == "__main__":
    # Create a SimulationManager instance and run the experiment
    experiment = SimulationManager()
    experiment.run_experiment(10) # Reduced number of runs for quicker testing
"""


'\nif __name__ == "__main__":\n    # Create a SimulationManager instance and run the experiment\n    experiment = SimulationManager()\n    experiment.run_experiment(10) # Reduced number of runs for quicker testing\n'

In [7]:
import random

class Individual:
    def __init__(self, distP=None, angleP=None):
        # Genes (se não dados, inicializar aleatoriamente)
        self.distP = distP if distP is not None else random.uniform(5.0, 15.0)
        self.angleP = angleP if angleP is not None else random.uniform(3.0, 10.0)
        self.fitness = None  # Fitness será avaliado depois

    def mutate(self, mutation_rate=0.1):
        """
        Pequena mutação nos genes (±10%).
        """
        if random.random() < mutation_rate:
            self.distP *= random.uniform(0.9, 1.1)
        if random.random() < mutation_rate:
            self.angleP *= random.uniform(0.9, 1.1)

    def crossover(self, other):
        """
        Cruzamento simples: média dos genes dos dois pais.
        """
        child_distP = (self.distP + other.distP) / 2.0
        child_angleP = (self.angleP + other.angleP) / 2.0
        return Individual(distP=child_distP, angleP=child_angleP)

    def get_genes(self):
        """|
        Devolve os genes (usado para passar ao robô durante a simulação).
        """
        return self.distP, self.angleP

    def __repr__(self):
        return f"Individual(distP={self.distP:.2f}, angleP={self.angleP:.2f}, fitness={self.fitness})"


In [8]:
class Population:
    def __init__(self, size, mutation_rate=0.1, elitism=1):
        self.size = size
        self.mutation_rate = mutation_rate
        self.elitism = elitism  # Número de melhores indivíduos a copiar diretamente
        self.individuals = [Individual() for _ in range(size)]

    def evaluate(self, simulator):
        """
        Avalia todos os indivíduos da população, usando o simulador fornecido.
        """
        for individual in self.individuals:
            distP, angleP = individual.get_genes()
            individual.fitness = simulator.run_with_parameters(distP, angleP)

    def select_parents(self):
        """
        Seleção por torneio: escolhe 2 indivíduos com melhor fitness.
        """
        tournament_size = 3
        parents = random.sample(self.individuals, tournament_size)
        parents.sort(key=lambda ind: ind.fitness, reverse=True)  # Fitness mais alto é melhor
        return parents[0], parents[1]

    def create_next_generation(self):
        """
        Cria a próxima geração através de elitismo + crossover + mutação.
        """
        # Ordena a população atual
        self.individuals.sort(key=lambda ind: ind.fitness, reverse=True)

        next_generation = []

        # Elitismo: copia os melhores
        for i in range(self.elitism):
            next_generation.append(self.individuals[i])

        # Gerar o resto da nova geração
        while len(next_generation) < self.size:
            parent1, parent2 = self.select_parents()
            child = parent1.crossover(parent2)
            child.mutate(mutation_rate=self.mutation_rate)
            next_generation.append(child)

        self.individuals = next_generation

    def get_best_individual(self):
        """
        Devolve o melhor indivíduo da população.
        """
        return max(self.individuals, key=lambda ind: ind.fitness)


In [9]:
import random

class GeneticOptimizer:
    def __init__(self, simulation_manager, population_size=20, generations=10, mutation_rate=0.1):
        """
        Genetic Algorithm for optimizing robot navigation parameters.
        """
        self.simulation_manager = simulation_manager
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate

    def _create_individual(self):
        """
        Creates a random individual [distP, angleP].
        """
        distP = random.uniform(5.0, 20.0)  # Range for proportional gain on distance
        angleP = random.uniform(2.0, 10.0)  # Range for proportional gain on angle
        return [distP, angleP]

    def _mutate(self, individual):
        """
        Mutates an individual by adding small random noise.
        """
        if random.random() < self.mutation_rate:
            individual[0] += random.uniform(-2.0, 2.0)  # Mutate distP
        if random.random() < self.mutation_rate:
            individual[1] += random.uniform(-1.0, 1.0)  # Mutate angleP
        return individual

    def _crossover(self, parent1, parent2):
        """
        Creates two children from two parents using one-point crossover.
        """
        if random.random() < 0.5:
            child1 = [parent1[0], parent2[1]]
            child2 = [parent2[0], parent1[1]]
        else:
            child1 = [parent2[0], parent1[1]]
            child2 = [parent1[0], parent2[1]]
        return child1, child2

    def optimize(self):
        """
        Runs the Genetic Algorithm to optimize parameters.
        """
        best_individual = None
        best_fitness = -float("inf")
    
        for generation in range(self.generations):
            print(f"\n=== Geração {generation + 1} ===")
            
            population = [self._create_individual() for _ in range(self.population_size)]
            fitness_scores = []
        
            for idx, individual in enumerate(population):
                try:
                    distP, angleP = individual
                    print(f"\n👤 Indivíduo {idx+1} → distP = {distP:.2f}, angleP = {angleP:.2f}")
                    average_fitness = self.simulation_manager.run_experiment_with_params(distP, angleP)
                    fitness_scores.append((average_fitness, individual))
        
                    if average_fitness > best_fitness:
                        best_fitness = average_fitness
                        best_individual = individual
                except Exception as e:
                    print(f"[ERRO] Problema ao avaliar o indivíduo {idx+1}: {e}")
            
            print(f"[INFO] Geração {generation + 1} concluída.")

    
        print(f"\n🏆 Melhor indivíduo: distP = {best_individual[0]:.3f}, angleP = {best_individual[1]:.3f} com fitness = {best_fitness:.2f}")
        return best_individual
    
    


In [10]:
from controller import Supervisor

# Função para configurar a dificuldade
def get_difficulty_settings(difficulty):
    if difficulty == 'easy':
        return 2, (math.radians(20), math.radians(30)), 4.0
    elif difficulty == 'hard':
        return 7, (math.radians(60), math.radians(100)), 2.0
    else:  # medium
        return 4, (math.radians(35), math.radians(60)), 3.0

if __name__ == "__main__":
    supervisor = Supervisor()  # cria só UM Supervisor
    experiment = SimulationManager(supervisor)  # passa o supervisor

    optimizer = GeneticOptimizer(
        experiment,
        population_size=10,
        generations=5,
        mutation_rate=0.2
    )

    best_params = optimizer.optimize()
    print(f"Best parameters found: distP = {best_params[0]:.3f}, angleP = {best_params[1]:.3f}")



=== Geração 1 ===

👤 Indivíduo 1 → distP = 7.07, angleP = 3.05

--- Simulação com dificuldade: EASY ---
Building tunnel with clearance factor: 4.00
⚡ [ALERTA] Robô saiu do túnel (easy): [-0.10763492 -0.13899381]
🎯 Fitness para easy: -420.94

--- Simulação com dificuldade: MEDIUM ---
Building tunnel with clearance factor: 3.00
⏰ TIME OUT
🎯 Fitness para medium: 38.79

--- Simulação com dificuldade: HARD ---
Building tunnel with clearance factor: 2.00
Skipping obstacle—too close to another.
⏰ TIME OUT
🎯 Fitness para hard: 16.62

🔚 Média final do fitness: -121.84

👤 Indivíduo 2 → distP = 18.97, angleP = 7.63

--- Simulação com dificuldade: EASY ---
Building tunnel with clearance factor: 4.00
⚡ [ALERTA] Robô saiu do túnel (easy): [-0.10786822 -0.13851796]
🎯 Fitness para easy: -405.16

--- Simulação com dificuldade: MEDIUM ---
Building tunnel with clearance factor: 3.00
⏰ TIME OUT
🎯 Fitness para medium: 145.19

--- Simulação com dificuldade: HARD ---
Building tunnel with clearance factor: 2

In [14]:
best_distP = 8.338
best_angleP = 5.328

fitness = experiment.run_experiment_with_params(best_distP, best_angleP)
print(f"🔍 Avaliação direta com os melhores parâmetros → Fitness: {fitness:.2f}")



--- Simulação com dificuldade: EASY ---
Building tunnel with clearance factor: 4.00
⚡ [ALERTA] Robô saiu do túnel (easy): [-0.11037253 -0.13750938]
🎯 Fitness para easy: -386.96

--- Simulação com dificuldade: MEDIUM ---
Building tunnel with clearance factor: 3.00
⏰ TIME OUT
🎯 Fitness para medium: 23.15

--- Simulação com dificuldade: HARD ---
Building tunnel with clearance factor: 2.00
Skipping obstacle—too close to another.
⚡ [ALERTA] Robô saiu do túnel (hard): [-0.07360005 -0.07541836]
⏰ TIME OUT
🎯 Fitness para hard: -1986.75

🔚 Média final do fitness: -783.52
🔍 Avaliação direta com os melhores parâmetros → Fitness: -783.52
